In [12]:
import pandas as pd
import yaml
from tabulate import tabulate

In [13]:
datasets = ['kuhar', 'motionsense', 'uci', 'wisdm', 'realworld_thigh', 'realworld_waist']
for dataset in datasets:
    with open(f'../execute_once_experiments/TVT_sb_tdom_no_reducer/scores/TVT_sb_no_reducer_{dataset}.yaml') as f:
        scores = yaml.load(f, Loader=yaml.FullLoader)
        print(dataset, scores['score'])

kuhar 0.7965277777777777
motionsense 0.8872881355932204
uci 0.8855072463768117
wisdm 0.7693990755007704
realworld_thigh 0.6959972394755003
realworld_waist 0.663233024691358


In [14]:
datasets = ['kuhar', 'motionsense', 'uci', 'wisdm', 'realworld_thigh', 'realworld_waist']
models = ['umap', 'ae', 'tae', 'convae', 'convtae']
percentages = [2.5, 5, 25, 50, 75, 100, 200]

data = []
new_data = {
    'dataset': None,
    'model': None,
    'percentage': None,
    'score': None,
    'knn': None,
    'rf': None,
    'svm': None
}
for dataset in datasets:
    for model in models:
        for percentage in percentages:
            filename = f'../execute_once_experiments/TVT_sb_best_found_2024/scores/TVT_sb_{model}_{dataset}_P{percentage}.yaml'
            with open(filename) as f:
                scores = yaml.load(f, Loader=yaml.FullLoader)
                new_data['dataset'] = dataset
                new_data['model'] = model
                new_data['percentage'] = percentage
                new_data['score'] = scores['score']
                new_data['knn'] = scores['KNN-5-accuracy (mean)']
                new_data['rf'] = scores['randomforest-100-accuracy (mean)']
                new_data['svm'] = scores['SVM-rbf-C1.0-accuracy (mean)']
                data.append(new_data.copy())

KeyError: 'KNN-5-accuracy (mean)'

In [ ]:
import os
os.curdir + "/Documents and Settings"
os.listdir(os.curdir)

['ae-vs-convae-kuhar.png',
 'ae-vs-convae-motionsense.png',
 'ae-vs-convae-realworld_thigh.png',
 'ae-vs-convae-realworld_waist.png',
 'ae-vs-convae-uci.png',
 'ae-vs-convae-wisdm.png',
 'ae-vs-tae-kuhar.png',
 'ae-vs-tae-motionsense.png',
 'ae-vs-tae-realworld_thigh.png',
 'ae-vs-tae-realworld_waist.png',
 'ae-vs-tae-uci.png',
 'ae-vs-tae-wisdm.png',
 'ae_kuhar',
 'ae_kuhar_gradual.png',
 'balanced vs balanced_user.ipynb',
 'compile_data.ipynb',
 'convae-vs-convtae-kuhar.png',
 'convae-vs-convtae-motionsense.png',
 'convae-vs-convtae-realworld_thigh.png',
 'convae-vs-convtae-realworld_waist.png',
 'convae-vs-convtae-uci.png',
 'convae-vs-convtae-wisdm.png',
 'data no reducer.ipynb',
 'explore_hyperparameters.ipynb',
 'exploring hyperparameters ae.ipynb',
 'exploring hyperparameters P10 fixed dataset.ipynb',
 'exploring hyperparameters P10 fixed percentage.ipynb',
 'exploring hyperparameters P10 histogramas.ipynb',
 'exploring hyperparameters P10 summary all.ipynb',
 'exploring hyperpa

In [ ]:
data = pd.DataFrame(data)
data = data[data['dataset'] == 'uci']
data

,dataset,model,percentage,score,knn,rf,svm
40,uci,umap,25,0.657391,0.649275,0.657391,0.617391
41,uci,umap,50,0.653768,0.652174,0.653768,0.649275
42,uci,umap,75,0.656667,0.650725,0.656667,0.560870
43,uci,umap,100,0.653623,0.653623,0.638261,0.579710
44,uci,ae,25,0.727536,0.675362,0.721594,0.727536
45,uci,ae,50,0.740580,0.692754,0.723768,0.740580
46,uci,ae,75,0.736232,0.714493,0.735507,0.736232
47,uci,ae,100,0.742029,0.742029,0.733768,0.737681
48,uci,tae,25,0.704783,0.573913,0.704783,0.704348
49,uci,tae,50,0.682609,0.530435,0.667681,0.682609


In [ ]:
data = pd.DataFrame(data)
# Drop rows with percentage 100 and 50
data = data[(data['percentage'] != 100) & (data['percentage'] != 50)]
# Add column with best string for each row
data['best'] = data[['knn', 'rf', 'svm']].idxmax(axis=1)
rows = []
for key, group in data.groupby(['model', 'percentage']):
    group.drop(['model', 'percentage', 'knn', 'rf', 'svm'], axis=1, inplace=True)
    # Round numbers
    
    group = group.set_index('dataset')
    group['score'] = group['score'].apply(lambda x: round(x*100, 1))
    group['VALUE'] = (group['score']).astype(str) + '%(' + group['best'].apply(str.upper) + ')'
    group = group.drop(['score', 'best'], axis=1)
    # print(group)
    group = group.T
    
    # print(group.loc['best'], '\nnnn')
    # Add key
    group['model'] = key[0]
    group['percentage'] = key[1]
    rows.append(group)
table3 = pd.concat(rows)
# Reorder columns
# print(tabulate(table3, headers='keys', tablefmt='psql'))
table3 = table3[['model', 'percentage', 'uci', 'motionsense', 'kuhar', 'wisdm', 'realworld_thigh', 'realworld_waist']]
table3 = table3.reset_index(drop=True)
# Change column names
table3 = table3.rename(columns={'uci': 'UCI', 'motionsense': 'MS', 'kuhar': 'KH', 'wisdm': 'WISDM', 'realworld_thigh': 'RW Thigh', 'realworld_waist': 'RW Waist'})
print(tabulate(table3, headers='keys', tablefmt='latex'))
# display(table3)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    


\begin{tabular}{rlrllllll}
\hline
    & model   &   percentage & UCI        & MS         & KH         & WISDM      & RW Thigh   & RW Waist   \\
\hline
  0 & ae      &           25 & 72.8\%(SVM) & 79.5\%(KNN) & 66.7\%(SVM) & 70.2\%(KNN) & 65.8\%(SVM) & 60.6\%(SVM) \\
  1 & ae      &           75 & 73.6\%(SVM) & 79.7\%(KNN) & 68.1\%(SVM) & 70.7\%(RF)  & 68.9\%(RF)  & 60.5\%(SVM) \\
  2 & convae  &           25 & 20.0\%(KNN) & 79.8\%(KNN) & 63.9\%(KNN) & 66.0\%(RF)  & 61.1\%(KNN) & 68.4\%(SVM) \\
  3 & convae  &           75 & 68.6\%(KNN) & 73.7\%(KNN) & 66.0\%(KNN) & 20.0\%(KNN) & 72.7\%(KNN) & 16.7\%(RF)  \\
  4 & convtae &           25 & 74.8\%(RF)  & 73.7\%(RF)  & 59.7\%(SVM) & 65.8\%(RF)  & 66.8\%(RF)  & 67.3\%(SVM) \\
  5 & convtae &           75 & 63.3\%(RF)  & 60.2\%(RF)  & 63.1\%(RF)  & 69.6\%(RF)  & 48.0\%(RF)  & 58.9\%(RF)  \\
  6 & tae     &           25 & 70.5\%(RF)  & 75.2\%(RF)  & 65.3\%(SVM) & 67.1\%(RF)  & 62.8\%(RF)  & 65.9\%(SVM) \\
  7 & tae     &           75 & 69.9\%

In [ ]:
data

,dataset,model,percentage,score,knn,rf,svm,best
0,kuhar,umap,25,0.590278,0.590278,0.579167,0.527778,knn
2,kuhar,umap,75,0.583333,0.583333,0.571528,0.534722,knn
4,kuhar,ae,25,0.666667,0.645833,0.626389,0.666667,svm
6,kuhar,ae,75,0.680556,0.659722,0.645139,0.680556,svm
8,kuhar,tae,25,0.652778,0.631944,0.611111,0.652778,svm
10,kuhar,tae,75,0.673611,0.673611,0.663194,0.604167,knn
12,kuhar,convae,25,0.638889,0.638889,0.577778,0.402778,knn
14,kuhar,convae,75,0.659722,0.659722,0.591667,0.576389,knn
16,kuhar,convtae,25,0.597222,0.541667,0.559028,0.597222,svm
18,kuhar,convtae,75,0.631250,0.576389,0.631250,0.583333,rf


# Fdom

In [ ]:
datasets = ['kuhar', 'motionsense', 'uci', 'wisdm', 'realworld_thigh', 'realworld_waist']
datasets = ['wisdm', 'realworld_thigh', 'realworld_waist']

data = []
new_data = {
    'dataset': None,
    'experiment': None,
    'knn': None,
    'rf': None,
    'svm': None,
    'score': None
}
experiments = ['TVT_sb_fdom_no_reducer', 'TVT_sb_fdom_acc_no_reducer', 'TVT_sb_fdom_gyr_no_reducer'] 
for dataset in datasets:
    for experiment in experiments:
        filename = f'../execute_once_experiments/{experiment}/scores/TVT_sb_no_reducer_{dataset}.yaml'
        with open(filename) as f:
            scores = yaml.load(f, Loader=yaml.FullLoader)
            new_data['dataset'] = dataset
            new_data['experiment'] = experiment
            new_data['score'] = scores['score']
            new_data['knn'] = scores['KNN-5-accuracy (mean)']
            new_data['rf'] = scores['randomforest-100-accuracy (mean)']
            new_data['svm'] = scores['SVM-rbf-C1.0-accuracy (mean)']
            data.append(new_data.copy())

In [ ]:
print(tabulate(pd.DataFrame(data), headers='keys', tablefmt='psql'))

+----+-----------------+----------------------------+----------+----------+----------+----------+
|    | dataset         | experiment                 |      knn |       rf |      svm |    score |
|----+-----------------+----------------------------+----------+----------+----------+----------|
|  0 | wisdm           | TVT_sb_fdom_no_reducer     | 0.862866 | 0.857134 | 0.804314 | 0.862866 |
|  1 | wisdm           | TVT_sb_fdom_acc_no_reducer | 0.832666 | 0.834977 | 0.771957 | 0.834977 |
|  2 | wisdm           | TVT_sb_fdom_gyr_no_reducer | 0.82681  | 0.829615 | 0.796302 | 0.829615 |
|  3 | realworld_thigh | TVT_sb_fdom_no_reducer     | 0.637336 | 0.802692 | 0.748102 | 0.802692 |
|  4 | realworld_thigh | TVT_sb_fdom_acc_no_reducer | 0.753278 | 0.792892 | 0.730504 | 0.792892 |
|  5 | realworld_thigh | TVT_sb_fdom_gyr_no_reducer | 0.542443 | 0.708213 | 0.704624 | 0.708213 |
|  6 | realworld_waist | TVT_sb_fdom_no_reducer     | 0.736497 | 0.738503 | 0.743827 | 0.743827 |
|  7 | realworld_wai